# 1

In [77]:
import pandas as pd

In [79]:
covid_data = pd.read_csv("COVID19_Dataset-text_labels_only.csv")

# 2

In [80]:
import nltk
from nltk.tokenize import word_tokenize

In [81]:
vocabulary = set()

for tweet in covid_data['Tweet']:
    tokens = word_tokenize(tweet)
    vocabulary.update(tokens)

vocabulary = sorted(vocabulary)

word_to_index = {word: idx for idx, word in enumerate(vocabulary)}

def one_hot_encode(tweet):
    tokens = word_tokenize(tweet)
    one_hot_vector = [0] * len(vocabulary)
    for token in tokens:
        if token in word_to_index:
            one_hot_vector[word_to_index[token]] = 1
    return one_hot_vector

In [82]:
covid_data['one_hot_encoded'] = covid_data['Tweet'].apply(one_hot_encode)

# 3

In [83]:
from collections import Counter
def bag_of_words(tweet):
    tokens = word_tokenize(tweet)
    word_counts = Counter(tokens)
    bow_vector = [0] * len(vocabulary)
    for word, count in word_counts.items():
        if word in word_to_index:
            bow_vector[word_to_index[word]] = count
    return bow_vector

covid_data['bow'] = covid_data['Tweet'].apply(bag_of_words)

# 4

In [84]:
from nltk.util import ngrams

In [85]:
ngram_vocab = set()
for tweet in covid_data['Tweet']:
    tokens = word_tokenize(tweet)
    n_grams = list(ngrams(tokens,2))
    ngram_vocab.update(n_grams)

ngram_w_2_idx = {ngram: idx for idx, ngram in enumerate(ngram_vocab)}

In [86]:
def bag_of_ngrams(tweet,n):
    tokens = word_tokenize(tweet)
    n_grams = list(ngrams(tokens,n))
    ngram_counts = Counter(n_grams)
    ngram_vector = [0] * len(ngram_vocab)
    for ngram, count in ngram_counts.items():
        if ngram in ngram_w_2_idx:
            ngram_vector[ngram_w_2_idx[ngram]] = count
    return ngram_vector

covid_data['bigrams'] = covid_data['Tweet'].apply(lambda x: bag_of_ngrams(x,n=2))

# 5

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [88]:
x_tfidf = vectorizer.fit_transform(covid_data['Tweet'])

x_tfidf_dense = x_tfidf.toarray()

tfidf_df = pd.DataFrame(x_tfidf_dense,columns=vectorizer.get_feature_names_out())

covid_data_tfidf = pd.concat([covid_data,tfidf_df],axis=1)